In [10]:
from bs4 import BeautifulSoup
import json
import requests
import random

import pandas as pd
import math, time
import re
import numpy as np
import logging
from requests.exceptions import Timeout

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
# pd.set_option("display.max_colwidth",None)

# TripAdvisor


In [11]:
# with open("user-agents_1.txt", "r+") as file:
#     user_agents = file.readlines()
#     user_agents = [user_agent.replace("\n","").strip() for user_agent in user_agents]

In [12]:
# JSON dosyasını oku
with open("data/user-agents.json", "r") as f:
    user_agents_data = json.load(f)

# User-Agent listesini oluştur
user_agents = [entry["ua"] for entry in user_agents_data]

In [13]:
# with open("valid-proxy-list.txt","r") as f:
#     prxs = f.read().split("\n")

# proxies =[]

# for prx in prxs:
#     if prx not in proxies:
#         proxies.append(prx)

In [51]:
def fetch_html(url):

    headers = {
        "User-Agent": random.choice(user_agents),
        "Accept-Language": "en-US,en;q=0.9",
        "Referer": "https://www.google.com/",
    }

    try:
        # proxy = random.choice(proxies)
        response = requests.get(url, headers=headers, timeout=10)
        response.raise_for_status()  
        return response.content
    except requests.exceptions.HTTPError as err:
        print(f"HTTP hatası: {err}")
        return None


def parse_html(html_content):
    return BeautifulSoup(html_content, 'lxml')


def get_namesandcount(soup):
    hotel_names_with_numbers = [hotel.get_text(strip=True) for hotel in soup.find_all(name="div", class_="nBrpc o W")]

    cleaned_hotel_names = [re.sub(r'[^\w\s]', '', re.sub(r'^\d+\.', '', hotel)) for hotel in hotel_names_with_numbers]

    review_counts = []
    for count_review in soup.find_all("span", class_="biGQs _P pZUbB hmDzD"):
        span_tag = count_review.find("span", class_="S4")
        if span_tag:
            review_counts.append(span_tag.text)

    review_counts_int = [int(re.sub(r'\.', '', re.search(r'\d+(\.\d+)*', review).group())) for review in review_counts]
    

    return pd.DataFrame({"name":cleaned_hotel_names, "count":review_counts_int })


def get_button_hrefs(soup):
    hrefs = []
    buttons = soup.find_all('button', class_='ypcsE _S wSSLS')
    
    for button in buttons:
        a_tag = button.find('a', href=True)
        if a_tag:
            hrefs.append(a_tag['href'])
    
    return hrefs

def get_dynamic_links(hotels):
    hotel_review_links = []
    root = "https://www.tripadvisor.com.tr"
    for hotel in hotels:
        part = hotel
        full= root+part
        # print(f"\nLinkler: {full}")
        
        location = full.find('Reviews')
        
        # print(f"Linkteki Reviewsin Konumu: {location}")
        full = full[:(location+7)] +"-or{}"+ full[(location+7):]
        # print(f"Dinamik Link: {full}")
        
        hotel_review_links.append(full)
    
    return hotel_review_links


def get_review_by_url(url):
    
    try:
        review_page = parse_html(fetch_html(url))
        reviews = review_page.find_all("div", class_="_T FKffI bmUTE")
        
        texts=[]
        for review in reviews:
            span_element = review.find("span", class_="orRIx Ci _a C")
            if span_element is not None:
                texts.append(span_element.text)
        
        return texts
    except :
        return " "

In [52]:
url = "https://www.tripadvisor.com.tr/Hotels-g298020-Bodrum_District_Mugla_Province_Turkish_Aegean_Coast-Hotels.html"
response = fetch_html(url=url)

In [53]:
soup = parse_html(response)

In [54]:
hotels = get_button_hrefs(soup)

In [55]:
hotels

['/Hotel_Review-g1467782-d9865057-Reviews-Titanic_Luxury_Collection_Bodrum-Guvercinlik_Bodrum_District_Mugla_Province_Turkish_A.html#REVIEWS',
 '/Hotel_Review-g298658-d737330-Reviews-Parkim_Ayaz_Hotel-Bodrum_City_Bodrum_District_Mugla_Province_Turkish_Aegean_Coast.html#REVIEWS',
 '/Hotel_Review-g951437-d1412957-Reviews-Kefaluka_Resort_Bodrum-Akyarlar_Turgutreis_Bodrum_District_Mugla_Province_Turkish_Aege.html#REVIEWS',
 '/Hotel_Review-g1467782-d6939253-Reviews-La_Blanche_Island_Bodrum-Guvercinlik_Bodrum_District_Mugla_Province_Turkish_Aegean_Co.html#REVIEWS',
 '/Hotel_Review-g298663-d507984-Reviews-Rixos_Premium_Bodrum-Torba_Bodrum_District_Mugla_Province_Turkish_Aegean_Coast.html#REVIEWS',
 '/Hotel_Review-g298663-d15355708-Reviews-Duja_Bodrum-Torba_Bodrum_District_Mugla_Province_Turkish_Aegean_Coast.html#REVIEWS',
 '/Hotel_Review-g298658-d643023-Reviews-Aegean_Gate_Hotel-Bodrum_City_Bodrum_District_Mugla_Province_Turkish_Aegean_Coast.html#REVIEWS',
 '/Hotel_Review-g298661-d514483-Revi

In [24]:
hotel_review_links = get_dynamic_links(hotels)

In [25]:
hotel_review_links

['https://www.tripadvisor.com.tr/Hotel_Review-g1467782-d9865057-Reviews-or{}-Titanic_Luxury_Collection_Bodrum-Guvercinlik_Bodrum_District_Mugla_Province_Turkish_A.html#REVIEWS',
 'https://www.tripadvisor.com.tr/Hotel_Review-g298658-d737330-Reviews-or{}-Parkim_Ayaz_Hotel-Bodrum_City_Bodrum_District_Mugla_Province_Turkish_Aegean_Coast.html#REVIEWS',
 'https://www.tripadvisor.com.tr/Hotel_Review-g951437-d1412957-Reviews-or{}-Kefaluka_Resort_Bodrum-Akyarlar_Turgutreis_Bodrum_District_Mugla_Province_Turkish_Aege.html#REVIEWS',
 'https://www.tripadvisor.com.tr/Hotel_Review-g1467782-d6939253-Reviews-or{}-La_Blanche_Island_Bodrum-Guvercinlik_Bodrum_District_Mugla_Province_Turkish_Aegean_Co.html#REVIEWS',
 'https://www.tripadvisor.com.tr/Hotel_Review-g298663-d507984-Reviews-or{}-Rixos_Premium_Bodrum-Torba_Bodrum_District_Mugla_Province_Turkish_Aegean_Coast.html#REVIEWS',
 'https://www.tripadvisor.com.tr/Hotel_Review-g298663-d15355708-Reviews-or{}-Duja_Bodrum-Torba_Bodrum_District_Mugla_Province

In [55]:
def get_turkish_review_counts(hotel_review_links):
    """Fetch Turkish review counts from a list of hotel review links."""
    review_counts = []

    for link in hotel_review_links:
        html_content = fetch_html(link)
        
        if html_content:
            rev_page = parse_html(html_content)
            turkish_label = rev_page.find('span', string='Türkçe')
            if turkish_label:
                turkish_review_count = turkish_label.find_next('span', class_='mUseN').text
                turkish_review_count = turkish_review_count.strip('()').replace('.', '')
                review_counts.append(int(turkish_review_count))
                print(turkish_review_count)
            else:
                logging.error('Turkish review count not found')
                review_counts.append(None)
        else:
            logging.error('Failed to fetch HTML content')
            review_counts.append(None)
    
    return review_counts

In [30]:
get_review_by_url(hotel_review_links[0].format(5))

['Otel konum olarak çok güzel temizlik süper personel çok ilgili bellboy  berat beye güzel karşılaması ve ilgisi için çok teşekkür etmek istiyorum teşekkürler okuyorsan bu yorumu çocukların berat abisi :) tek kötü yanı yamaç oteli olması birazcık yokuşlu',
 'Otele arkadaslarimizla geldik. Gercekten harika bir ortam var aktiviteler dolu doku geciyor. Yemekler cok lezzetli ve cok fazla cesit var gercekten hem goze hem lezzete hitap ediyor. Sahili tertemiz rahatlikla yuzebilirsiniz. Yamas restoranin konsepti de cok harikaydi. Ayrica bizimle ilgilenen animasyon ekibinden Mert e cok tesekkur ederiz. Voleybolda gizem ve yulia nin enerjisi de gercekten harikaydi. Ozkan Bey ve Ugur Bey gercekten harika bir animasyon takimi kurmus herkese tesekkurler.',
 'Sahili, ilgili personeli ve yemekleriyle güzel bir tatil deneyimi yaşadık. Tavsiye ediyoruz kesinlikle, animasyon  ekibinden Mert beye de özel ilgi alakası için teşekkür ediyorum. Tekrar geleceğimiz bidahaki tatili dört gözle bekliyoruz',
 'Ge

In [26]:
hotel_df = get_namesandcount(soup)
hotel_df["review_link"] = hotel_review_links

In [27]:
results = []
for idx, row in hotel_df.iterrows():
    links = []
    for rng in range(0, row["count"], 5):
        if rng ==0: continue
        link = row["review_link"].format(rng)
        links.append(link)
    results.append(links)

hotel_df["dynamic_links"] = results

In [28]:
hotel_df

,name,count,review_link,dynamic_links
0,Titanic Luxury Collection Bodrum,5866,https://www.tripadvisor.com.tr/Hotel_Review-g1...,[https://www.tripadvisor.com.tr/Hotel_Review-g...
1,Parkim Ayaz Otel,2918,https://www.tripadvisor.com.tr/Hotel_Review-g2...,[https://www.tripadvisor.com.tr/Hotel_Review-g...
2,Kefaluka Resort Bodrum,3273,https://www.tripadvisor.com.tr/Hotel_Review-g9...,[https://www.tripadvisor.com.tr/Hotel_Review-g...
3,La Blanche Island Bodrum,3877,https://www.tripadvisor.com.tr/Hotel_Review-g1...,[https://www.tripadvisor.com.tr/Hotel_Review-g...
4,Rixos Premium Bodrum,5042,https://www.tripadvisor.com.tr/Hotel_Review-g2...,[https://www.tripadvisor.com.tr/Hotel_Review-g...
5,Duja Bodrum,3312,https://www.tripadvisor.com.tr/Hotel_Review-g2...,[https://www.tripadvisor.com.tr/Hotel_Review-g...
6,Aegean Gate Hotel,401,https://www.tripadvisor.com.tr/Hotel_Review-g2...,[https://www.tripadvisor.com.tr/Hotel_Review-g...
7,Selectum Colours Bodrum,3801,https://www.tripadvisor.com.tr/Hotel_Review-g2...,[https://www.tripadvisor.com.tr/Hotel_Review-g...
8,Kairaba Bodrum Imperial,3181,https://www.tripadvisor.com.tr/Hotel_Review-g9...,[https://www.tripadvisor.com.tr/Hotel_Review-g...
9,El Vino Hotel Suites,1324,https://www.tripadvisor.com.tr/Hotel_Review-g2...,[https://www.tripadvisor.com.tr/Hotel_Review-g...


In [29]:
hotel_df["count"].sum()/5/2

6250.7

In [30]:
# hotel_df.to_csv("data/hotels_with_links2.csv", index=False)

# türkçe yorumları seç

In [31]:
import pandas as pd 

hotel_df = pd.read_csv("data/hotels_with_links2.csv")

In [32]:
[link for link in hotel_df["review_link"]]

['https://www.tripadvisor.com.tr/Hotel_Review-g1467782-d9865057-Reviews-or{}-Titanic_Luxury_Collection_Bodrum-Guvercinlik_Bodrum_District_Mugla_Province_Turkish_A.html#REVIEWS',
 'https://www.tripadvisor.com.tr/Hotel_Review-g298658-d737330-Reviews-or{}-Parkim_Ayaz_Hotel-Bodrum_City_Bodrum_District_Mugla_Province_Turkish_Aegean_Coast.html#REVIEWS',
 'https://www.tripadvisor.com.tr/Hotel_Review-g951437-d1412957-Reviews-or{}-Kefaluka_Resort_Bodrum-Akyarlar_Turgutreis_Bodrum_District_Mugla_Province_Turkish_Aege.html#REVIEWS',
 'https://www.tripadvisor.com.tr/Hotel_Review-g1467782-d6939253-Reviews-or{}-La_Blanche_Island_Bodrum-Guvercinlik_Bodrum_District_Mugla_Province_Turkish_Aegean_Co.html#REVIEWS',
 'https://www.tripadvisor.com.tr/Hotel_Review-g298663-d507984-Reviews-or{}-Rixos_Premium_Bodrum-Torba_Bodrum_District_Mugla_Province_Turkish_Aegean_Coast.html#REVIEWS',
 'https://www.tripadvisor.com.tr/Hotel_Review-g298663-d15355708-Reviews-or{}-Duja_Bodrum-Torba_Bodrum_District_Mugla_Province

In [33]:
turk_review_count =[3549,650,986,2065,2794,2107,22,768,1283,172,1065,1464,632,289,416,259,602,804,
                   102,73,2152,220,375,56,2311,1071,118,38,8,34]

In [34]:
len(turk_review_count), sum(turk_review_count)

(30, 26485)

In [35]:
hotel_df["turk_review"] = turk_review_count

In [36]:
import ast
hotel_df["dynamic_links"] = hotel_df["dynamic_links"].apply(ast.literal_eval)

In [37]:
[link for link in hotel_df["dynamic_links"][0]]

['https://www.tripadvisor.com.tr/Hotel_Review-g1467782-d9865057-Reviews-or5-Titanic_Luxury_Collection_Bodrum-Guvercinlik_Bodrum_District_Mugla_Province_Turkish_A.html#REVIEWS',
 'https://www.tripadvisor.com.tr/Hotel_Review-g1467782-d9865057-Reviews-or10-Titanic_Luxury_Collection_Bodrum-Guvercinlik_Bodrum_District_Mugla_Province_Turkish_A.html#REVIEWS',
 'https://www.tripadvisor.com.tr/Hotel_Review-g1467782-d9865057-Reviews-or15-Titanic_Luxury_Collection_Bodrum-Guvercinlik_Bodrum_District_Mugla_Province_Turkish_A.html#REVIEWS',
 'https://www.tripadvisor.com.tr/Hotel_Review-g1467782-d9865057-Reviews-or20-Titanic_Luxury_Collection_Bodrum-Guvercinlik_Bodrum_District_Mugla_Province_Turkish_A.html#REVIEWS',
 'https://www.tripadvisor.com.tr/Hotel_Review-g1467782-d9865057-Reviews-or25-Titanic_Luxury_Collection_Bodrum-Guvercinlik_Bodrum_District_Mugla_Province_Turkish_A.html#REVIEWS',
 'https://www.tripadvisor.com.tr/Hotel_Review-g1467782-d9865057-Reviews-or30-Titanic_Luxury_Collection_Bodrum-G

In [38]:
hotel_df.head()

,name,count,review_link,dynamic_links,turk_review
0,Titanic Luxury Collection Bodrum,5866,https://www.tripadvisor.com.tr/Hotel_Review-g1...,[https://www.tripadvisor.com.tr/Hotel_Review-g...,3549
1,Parkim Ayaz Otel,2918,https://www.tripadvisor.com.tr/Hotel_Review-g2...,[https://www.tripadvisor.com.tr/Hotel_Review-g...,650
2,Kefaluka Resort Bodrum,3273,https://www.tripadvisor.com.tr/Hotel_Review-g9...,[https://www.tripadvisor.com.tr/Hotel_Review-g...,986
3,La Blanche Island Bodrum,3877,https://www.tripadvisor.com.tr/Hotel_Review-g1...,[https://www.tripadvisor.com.tr/Hotel_Review-g...,2065
4,Rixos Premium Bodrum,5042,https://www.tripadvisor.com.tr/Hotel_Review-g2...,[https://www.tripadvisor.com.tr/Hotel_Review-g...,2794


In [39]:
def ceil_to_nearest_five(number):
    """Return the ceiling of number to the nearest multiple of 5."""
    remainder = number % 5
    if remainder == 0:
        return int(number)
    else:
        return int(number + (5 - remainder))

# Test cases
print(ceil_to_nearest_five(4.2))  # Output: 5
print(ceil_to_nearest_five(9.999))  # Output: 10
print(ceil_to_nearest_five(-3.7))  # Output: 0 (since -3.7 rounded up to nearest 5 is 0)
print(ceil_to_nearest_five(5.0))  # Output: 5
print(ceil_to_nearest_five(15))  # Output: 15
print(ceil_to_nearest_five(16))  # Output: 20


5
10
0
5
15
20


In [41]:
hotel_df["turk_review"] = hotel_df["turk_review"].apply(ceil_to_nearest_five)

In [42]:
import re

def extract_and_filter_urls(urls, user_number):
    """Extract the number from URLs after '-or' and return those with the number smaller than user_number."""
    filtered_urls = []
    
    for url in urls:
        match = re.search(r'-or(\d+)-', url)
        if match:
            number = int(match.group(1))
            if number <= user_number:
                filtered_urls.append(url)
            if user_number<5:
                filtered_urls.append(url)
    
    return filtered_urls


In [43]:
filtered_links=[]
for idx, row in hotel_df.iterrows():
    filtered_links.append(extract_and_filter_urls(row["dynamic_links"], row["turk_review"]))

In [44]:
hotel_df["filtered_links"] = filtered_links

In [48]:
hotel_df["filtered_links"].agg(len)

/tmp/ipykernel_368833/2452043133.py:1: FutureWarning: using <built-in function len> in Series.agg cannot aggregate and has been deprecated. Use Series.transform to keep behavior unchanged.
  hotel_df["filtered_links"].agg(len)


0     710
1     130
2     198
3     413
4     559
5     422
6       5
7     154
8     257
9      35
10    213
11    293
12    127
13     58
14     84
15     52
16    121
17    161
18     21
19     15
20    431
21     44
22     75
23     12
24    463
25    215
26     24
27      8
28      2
29      7
Name: filtered_links, dtype: int64

In [49]:
# hotel_df.to_csv("data/filtered_hotel_review_links2.csv", index=False)

# Scrape reviews

In [27]:
hotel_df = pd.read_csv("hotels_with_links.csv")

In [28]:
text_list = []
for idx, row in hotel_df.iterrows():
    text= []
    for i,url in enumerate(row["dynamic_links"]):
        print(f"Hotel Name: {row['name']} Count: {(i+1)*5}")
        print(url)
        print("----------------------------------------------")
        txt = get_review_by_url(url)
        time.sleep(5)
        text.extend(txt)
    
    print(f"New hotel\n\n")
    text_list.append(text)

Hotel Name: Concorde De Luxe Resort Count: 5
[
----------------------------------------------
Hotel Name: Concorde De Luxe Resort Count: 10
'
----------------------------------------------
Hotel Name: Concorde De Luxe Resort Count: 15
h
----------------------------------------------
Hotel Name: Concorde De Luxe Resort Count: 20
t
----------------------------------------------
Hotel Name: Concorde De Luxe Resort Count: 25
t
----------------------------------------------
Hotel Name: Concorde De Luxe Resort Count: 30
p
----------------------------------------------


KeyboardInterrupt: 

In [ ]:
text

['Concorde Otel, mükemmel hizmet kalitesi ve konforuyla öne çıkıyor. Şık ve modern odaları, lezzetli yemekler sunan restoranları ve güler yüzlü personeli ile misafirlerine unutulmaz bir deneyim yaşatıyor. Merkezi konumu sayesinde şehirdeki önemli noktalara kolayca ulaşım imkanı sunuyor.',
 'Buraya geldiğim için çok mutluyum buradaki her şeyi genel anlamda çok beğendim yemekleri çok lezzetli sahili plajı denizi havuzu herşeyiyle mükemmel çalışanlar güler yüzlü ve ilgili yemekler çok lezzetliydi özellikle sahildeki Boyan partiye bayıldım animasyon dans yolları gece etkinlikleri canlı müziği herşeyiyle kesinlikle gelmeye değer kesinlikle ailemle tekrar geleceğim',
 'Herkes cok kibar ve iyi.. Temizlik çok güzel..Restaurant harika yemekler çok lezzetli.. Burada kötü bir ṣey hiç görmedim..Denizi havuzu aqua parkì harika.. Harika dans shovları var.. Aktiviteler harika.. Voleybol, dart,yoga, step aerobik gibi harika aktiviteler var.. Canlı müzik ve dj performans gerçekten harika.. Çalisanlar ç

# Proxyy

In [64]:
import requests
from bs4 import BeautifulSoup
import random
import logging

logging.basicConfig(level=logging.INFO)

def get_html(link, **kwargs):
    """Fetch HTML content from a URL using a proxy."""
    max_retries = 5
    retries = 0

    while retries < max_retries:
        try:
            proxy = proxy_generator()
            response = requests.get(link, proxies=proxy, timeout=7, **kwargs)
            response.raise_for_status()
            source = BeautifulSoup(response.content, "html.parser")
            return source
        except requests.exceptions.RequestException as e:
            logging.error(f"Request error: {e}")
        except Exception as e:
            logging.error(f"An unexpected error occurred: {e}")
        retries += 1
        logging.info(f"Retrying... ({retries}/{max_retries})")
    
    logging.error(f"Failed to fetch HTML after {max_retries} retries.")
    return None

def proxy_generator():
    """Generate a random proxy from sslproxies.org."""
    response = requests.get("https://sslproxies.org/")
    soup = BeautifulSoup(response.content, 'html.parser')
    proxies = soup.find_all("tbody")[0].find_all("tr")
    proxies_array = []

    for i in proxies:
        try:
            ip = i.find_all("td")[0].text
            port = i.find_all("td")[1].text
            proxies_array.append({"https": f"http://{ip}:{port}"})
        except Exception as e:
            logging.error(f"Error parsing proxy: {e}")

    if proxies_array:
        return random.choice(proxies_array)
    else:
        logging.error("No valid proxies found.")
        return None


In [65]:
get_html('https://www.tripadvisor.com.tr/Hotel_Review-g20116893-d572784-Reviews-or5-Concorde_De_Luxe_Resort-Kemeragzi_Antalya_Turkish_Mediterranean_Coast.html#REVIEWS')

ERROR:root:Proxy check error: HTTPConnectionPool(host='79.137.194.203', port=4019): Max retries exceeded with url: http://www.google.com/ (Caused by ProxyError('Unable to connect to proxy', NewConnectionError('<urllib3.connection.HTTPConnection object at 0x71be12d1dea0>: Failed to establish a new connection: [Errno 111] Connection refused')))
ERROR:root:Proxy check error: HTTPConnectionPool(host='203.189.88.156', port=80): Read timed out. (read timeout=5)
ERROR:root:Proxy check error: HTTPConnectionPool(host='51.145.176.250', port=8080): Max retries exceeded with url: http://www.google.com/ (Caused by ProxyError('Unable to connect to proxy', ConnectTimeoutError(<urllib3.connection.HTTPConnection object at 0x71be12d1e080>, 'Connection to 51.145.176.250 timed out. (connect timeout=5)')))
ERROR:root:Proxy check error: HTTPConnectionPool(host='178.128.113.118', port=23128): Read timed out. (read timeout=5)
ERROR:root:Proxy check error: HTTPConnectionPool(host='203.205.9.105', port=8080): R

KeyboardInterrupt: 

In [67]:
import json
import random
import re
import time
import ast
import requests
import pandas as pd
from bs4 import BeautifulSoup as bs
import logging
from requests.exceptions import Timeout
def fetch_html(url, user_agents):
    """Fetch HTML content from a URL using random user agents."""
    headers = {
        "User-Agent": random.choice(user_agents),
        "Accept-Language": "en-US,en;q=0.9",
        "Referer": "https://www.google.com/",
    }
    max_retries = 5
    retries = 0
    while retries < max_retries:
        try:
            proxy = proxy_generator()
            response = requests.get(url=url, proxies=proxy, headers=headers, timeout=10)
            logging.info(f"Status Code: {response.status_code}")
            response.raise_for_status()
            return response.content
        except:
            logging.error(f"Error fetching HTML from URL: {url}")
        retries += 1
    logging.error(f"Failed to fetch HTML after {max_retries} retries.")
    return None

def get_user_agents(path):
    """Load user agents from a JSON file."""
    with open(path, "r", encoding="utf-8") as f:
        user_agents_data = json.load(f)
    return [entry["ua"] for entry in user_agents_data]

def parse_html(content):
    """Parse HTML content using BeautifulSoup."""
    if content:
        return bs(content, "lxml")
    logging.error(f"Error parsing HTML")
    return None


In [68]:
user_agents = get_user_agents("data/user-agents.json")

In [69]:
url = "https://google.com"


In [ ]:
fetch_html(url,user_agents)

ERROR:root:Proxy check error: HTTPConnectionPool(host='79.137.194.203', port=4019): Max retries exceeded with url: http://www.google.com/ (Caused by ProxyError('Unable to connect to proxy', NewConnectionError('<urllib3.connection.HTTPConnection object at 0x71be12e23dc0>: Failed to establish a new connection: [Errno 111] Connection refused')))
ERROR:root:Proxy check error: ('Connection broken: IncompleteRead(3130 bytes read, 6078 more expected)', IncompleteRead(3130 bytes read, 6078 more expected))
ERROR:root:Proxy check error: HTTPConnectionPool(host='203.189.88.156', port=80): Max retries exceeded with url: http://www.google.com/ (Caused by ProxyError('Unable to connect to proxy', ConnectTimeoutError(<urllib3.connection.HTTPConnection object at 0x71be12e237f0>, 'Connection to 203.189.88.156 timed out. (connect timeout=5)')))
ERROR:root:Proxy check error: HTTPConnectionPool(host='64.112.184.89', port=3128): Read timed out. (read timeout=5)
ERROR:root:Proxy check error: HTTPConnectionPo

In [ ]:
def is_proxy_working(proxy):
    """Check if the given proxy is working by sending a request through it."""
    try:
        response = requests.get("http://www.google.com", proxies=proxy, timeout=5)
        return response.status_code == 200
    except Exception as e:
        logging.error(f"Proxy check error: {e}")
        return False

def proxy_generator():
    """Generate a random proxy from sslproxies.org and verify if it's working."""
    response = requests.get("https://sslproxies.org/")
    soup = BeautifulSoup(response.content, 'html.parser')
    proxies = soup.find_all("tbody")[0].find_all("tr")
    proxies_array = []

    for i in proxies:
        try:
            ip = i.find_all("td")[0].text
            port = i.find_all("td")[1].text
            proxy = {"http": f"http://{ip}:{port}", "https": f"http://{ip}:{port}"}
            if is_proxy_working(proxy):
                proxies_array.append(proxy)
        except Exception as e:
            logging.error(f"Error parsing proxy: {e}")

    if proxies_array:
        return random.choice(proxies_array)
    else:
        logging.error("No valid proxies found.")
        return None

# create df

In [7]:
import json
import random
import re
import time
import ast
import requests
import pandas as pd
from bs4 import BeautifulSoup as bs
import logging
from requests.exceptions import Timeout

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

def get_user_agents(path):
    """Load user agents from a JSON file."""
    with open(path, "r", encoding="utf-8") as f:
        user_agents_data = json.load(f)
    return [entry["ua"] for entry in user_agents_data]
    
def fetch_html(url, user_agents):
    """Fetch HTML content from a URL using random user agents."""
    headers = {
        "User-Agent": random.choice(user_agents),
        "Accept-Language": "en-US,en;q=0.9",
        "Referer": "https://www.google.com/",
    }
    max_retries = 35
    retries = 0
    while retries < max_retries:
        try:
            proxy = proxy_generator()
            response = requests.get(url=url, proxies=proxy, headers=headers, timeout=20)
            logging.info(f"Status Code: {response.status_code}")
            response.raise_for_status()
            return response.content
        except:
            logging.error(f"Error fetching HTML from URL: {url}")
        retries += 1
    logging.error(f"Failed to fetch HTML after {max_retries} retries.")
    return None
    
def get_button_hrefs(soup):
    """Extract href links from button elements in the parsed HTML."""
    hrefs = []
    buttons = soup.find_all('button', class_='ypcsE _S wSSLS')
    for button in buttons:
        a_tag = button.find('a', href=True)
        if a_tag:
            hrefs.append(a_tag['href'])
    return hrefs
    
def get_dynamic_links(hotels):
    """Generate dynamic review links for hotels."""
    root = "https://www.tripadvisor.com.tr"
    hotel_review_links = [root + hotel for hotel in hotels]
    return [
        link[:link.find('Reviews') + 7] + "-or{}" + link[link.find('Reviews') + 7:]
        for link in hotel_review_links
    ]

def get_names_and_count(soup):
    """Extract hotel names and review counts from the parsed HTML."""
    hotel_names_with_numbers = [
        hotel.get_text(strip=True) for hotel in soup.find_all(name="div", class_="nBrpc o W")
    ]
    cleaned_hotel_names = [
        re.sub(r'[^\w\s]', '', re.sub(r'^\d+\.', '', hotel)) for hotel in hotel_names_with_numbers
    ]

    review_counts = []
    for count_review in soup.find_all("span", class_="biGQs _P pZUbB hmDzD"):
        span_tag = count_review.find("span", class_="S4")
        if span_tag:
            review_counts.append(span_tag.text)

    review_counts_int = [
        int(re.sub(r'\.', '', re.search(r'\d+(\.\d+)*', review).group())) for review in review_counts
    ]

    return pd.DataFrame({"name": cleaned_hotel_names, "count": review_counts_int})

def get_all_dynamic_links(hotel_df):
    """Generate all dynamic review links for a dataframe of hotels."""
    results = []
    for _, row in hotel_df.iterrows():
        links = [
            row["review_link"].format(rng)
            for rng in range(5, row["count"], 5)
        ]
        results.append(links)
    return results


def create_hotel_df(user_agents_path, output_path):
    """Create a dataframe of hotels with their dynamic review links."""
    user_agents = get_user_agents(user_agents_path)
    response = fetch_html(url=HOTELS_URL, user_agents=user_agents)

    if response:
        hotels = get_button_hrefs(parse_html(response))
        hotel_review_links = get_dynamic_links(hotels)
        hotel_df = get_names_and_count(parse_html(response))
        hotel_df["review_link"] = hotel_review_links
        hotel_df["dynamic_links"] = get_all_dynamic_links(hotel_df)
        return hotel_df
        # hotel_df.to_csv(output_path, index=False)
    
    else:
        logging.error("Failed to fetch hotel data.")



In [8]:
user_agents_path = "data/user-agents.json"
output_path = None
HOTELS_URL = "https://www.tripadvisor.com.tr/Hotels-g297962-Antalya_Turkish_Mediterranean_Coast-Hotels.html"

In [9]:
create_hotel_df(user_agents_path, output_path)

2024-06-02 12:19:54,444 - ERROR - Error fetching HTML from URL: https://www.tripadvisor.com.tr/Hotels-g297962-Antalya_Turkish_Mediterranean_Coast-Hotels.html
2024-06-02 12:19:54,445 - ERROR - Error fetching HTML from URL: https://www.tripadvisor.com.tr/Hotels-g297962-Antalya_Turkish_Mediterranean_Coast-Hotels.html
2024-06-02 12:19:54,445 - ERROR - Error fetching HTML from URL: https://www.tripadvisor.com.tr/Hotels-g297962-Antalya_Turkish_Mediterranean_Coast-Hotels.html
2024-06-02 12:19:54,446 - ERROR - Error fetching HTML from URL: https://www.tripadvisor.com.tr/Hotels-g297962-Antalya_Turkish_Mediterranean_Coast-Hotels.html
2024-06-02 12:19:54,446 - ERROR - Error fetching HTML from URL: https://www.tripadvisor.com.tr/Hotels-g297962-Antalya_Turkish_Mediterranean_Coast-Hotels.html
2024-06-02 12:19:54,446 - ERROR - Error fetching HTML from URL: https://www.tripadvisor.com.tr/Hotels-g297962-Antalya_Turkish_Mediterranean_Coast-Hotels.html
2024-06-02 12:19:54,447 - ERROR - Error fetching HTM